In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pycaret

In [ ]:
df = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.corr()

In [ ]:
df.describe()

In [ ]:
data = df.sample(frac = 0.9, random_state = 42)
data_unseen = df.drop(data.index)

data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

In [ ]:
from pycaret.classification import *

In [ ]:
clf = setup(data = data, target ='DEATH_EVENT', session_id = 123,
           normalize = True, transformation = True,ignore_low_variance = True,
        remove_multicollinearity = True, multicollinearity_threshold = 0.95)

In [ ]:
compare_models()

In [ ]:
best_model = automl()

In [ ]:
best_model

In [ ]:
et = create_model('et', fold = 5)


In [ ]:
tuned_et = tune_model(et, optimize ='AUC')

In [ ]:
plot_model(tuned_et, plot = 'auc')


In [ ]:

plot_model(tuned_et, plot='feature')


In [ ]:
plot_model(tuned_et
           , plot = 'confusion_matrix')


In [ ]:
predict_model(tuned_et)

In [ ]:
final_et = finalize_model(tuned_et)

In [ ]:
unseen_predictions = predict_model(final_et, data=data_unseen)
unseen_predictions.head(10)


In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['DEATH_EVENT'], unseen_predictions['Label'], metric = 'Accuracy')

In [ ]:
save_model(final_et,'Final Extra Trees Model 25Apr2021')
